In [6]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time


def extract_chart_data(driver):
    html_source = driver.page_source
    parse = BeautifulSoup(html_source, 'html.parser')
    titles = parse.find_all("div", {"class": "ellipsis rank01"})
    singers = parse.find_all("div", {"class": "ellipsis rank02"})
    albums = parse.find_all("div", {"class": "ellipsis rank03"})
    dates = driver.find_element(By.CLASS_NAME, 'datelk').text
    likes_elements = parse.find_all("span", {"class": "cnt"})

    title_list = [title.find('a').text for title in titles]
    singer_list = [singer.find('span', {"class": "checkEllipsis"}).text for singer in singers]
    album_list = [album.find('a').text for album in albums]
    like_list = [re.sub(r'\D', '', like.text.replace("총건수", "").strip()) for like in likes_elements]
    date_list = [dates] * len(title_list)
    
    return title_list, singer_list, album_list, date_list, like_list

def main():
    URL = "https://www.melon.com/chart/index.htm"
    option = webdriver.ChromeOptions()
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
    driver.get(URL)
    title =[]
    album = []
    singer = []
    date = []
    like = []
    rank = []
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span')))
    except TimeoutException:
        print("페이지 로딩 시간 초과")
        driver.quit()
        return

    finder_tab = driver.find_element(By.XPATH,'//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span')
    finder_tab.click()
    time.sleep(1)
    
    weekly_chart = driver.find_element(By.XPATH,'//*[@id="d_chart_search"]/div/h4[1]')
    weekly_chart.click()
    time.sleep(1)

    # 메세지 확인용 변수
    decade_mapping = {1: '2020년대', 2: '2010년대'}
    year_offset = {1: (2024, -1), 2: (2019, -1)}  # (시작 연도, 증감량)
    for years in range(1, 3):
        decade_text = decade_mapping.get(years, "알 수 없는 년대")
        try:
            years_btn = driver.find_element(By.XPATH,f'//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[{years}]')
            years_btn.click()
            time.sleep(1)
        except NoSuchElementException:
            print(f"{decade_text} 구간의 데이터가 존재하지 않습니다.")
            continue

        # 연도선택
        start_year, year_change = year_offset[years]
        if years == 1 : # 2020년대
            for year_index in range(1, 6): # 2024,2023,2022,2021,2020 년대만 조회.
                year = start_year + (year_index - 1) * year_change
                try:
                    year_btn = driver.find_element(By.XPATH,f'//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[{year_index}]')
                    year_btn.click()
                    time.sleep(1)
                except NoSuchElementException:
                    print(f"{decade_text} 구간의 {year}년 데이터가 존재하지 않습니다.")
                    continue
                # 월간선택 ( 봄 : 3,4,5 겨울 1,2,12)
                target_month = [1,2,3,4,5,12]
                for month in target_month:
                    try:
                        month_btn = driver.find_element(By.XPATH,f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{month}]')
                        month_btn.click()
                        time.sleep(1)
                    except NoSuchElementException:
                        print(f"{year}년 {month}월 데이터가 존재하지 않습니다.")
                        continue
                    
                    # 주간선택
                    for week in range(1, 6):
                        try:
                            week_btn = driver.find_element(By.XPATH,f'//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[{week}]')
                            week_btn.click()                          
                            time.sleep(1)
                        except NoSuchElementException:
                            print(f"{year}년 {month}월 {week}주차 데이터가 존재하지 않습니다.")
                            break  # 해당 월의 주차가 더 이상 존재하지 않으므로 더 이상의 반복을 중단합니다.
                        
                        # 장르선택, 이 부분
                        try:
                            genre_btn = driver.find_element(By.XPATH,"""//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]""")
                            genre_btn.click()
                            search_btn = driver.find_element(By.XPATH,"""//*[@id="d_srch_form"]/div[2]/button""")
                            search_btn.click() 
                            time.sleep(1)
                            html_source = driver.page_source

                            parse = BeautifulSoup(html_source, 'html.parser')

                            titles = parse.find_all("div", {"class": "ellipsis rank01"}) 
                            singers = parse.find_all("div", {"class": "ellipsis rank02"}) 
                            albums = parse.find_all("div",{"class": "ellipsis rank03"}) 
                            dates = driver.find_element(By.CLASS_NAME,'datelk').text

                            likes_elements = parse.find_all("span", {"class": "cnt"})  # 좋아요 수 추출
                            likes = [element.text.strip() for element in likes_elements]


                            rank = [i for i in range(1,101)]


                            for t in titles:
                                title.append(t.find('a').text) 
                            for s in singers:
                                singer.append(s.find('span', {"class": "checkEllipsis"}).text)

                            for a in albums:
                                album.append(a.find('a').text)
                                date.append(dates)
                            for l in likes:
                                l = (re.sub(r'\D', '', l.replace("총건수", "").strip()))
                                like.append(l)

                            
                            time.sleep(1)
                            
                        except NoSuchElementException:
                            print(f"{year}년 {month}월 {week}주차의 장르종합 데이터가 존재하지 않습니다.")
                            continue
        else : 
            for year_index in range(1, 11): # 2019, 2018, ... 2010 년대 조회.
                year = start_year + (year_index - 1) * year_change
                try:
                    year_btn = driver.find_element(By.XPATH,f'//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[{year_index}]')
                    year_btn.click()
                    time.sleep(1)
                except NoSuchElementException:
                    print(f"{decade_text} 구간의 {year}년 데이터가 존재하지 않습니다.")
                    continue

                # 월간선택 ( 봄 : 3,4,5 겨울 1,2,12)
                target_month = [1,2,3,4,5,12]
                for month in target_month:
                    try:
                        month_btn = driver.find_element(By.XPATH,f'//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[{month}]')
                        month_btn.click()
                        time.sleep(1)
                    except NoSuchElementException:
                        print(f"{year}년 {month}월 데이터가 존재하지 않습니다.")
                        continue

                    # 주간선택
                    for week in range(1, 6):
                        try:
                            week_btn = driver.find_element(By.XPATH,f'//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[{week}]')
                            week_btn.click()                          
                            time.sleep(1)
                        except NoSuchElementException:
                            print(f"{year}년 {month}월 {week}주차 데이터가 존재하지 않습니다.")
                            break  # 해당 월의 주차가 더 이상 존재하지 않으므로 더 이상의 반복을 중단합니다.

                        # 장르선택
                        try:
                            genre_btn = driver.find_element(By.XPATH,"""//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]""")
                            genre_btn.click()
                            search_btn = driver.find_element(By.XPATH,"""//*[@id="d_srch_form"]/div[2]/button""")
                            search_btn.click()
                            time.sleep(1)
                            html_source = driver.page_source

                            parse = BeautifulSoup(html_source, 'html.parser')

                            titles = parse.find_all("div", {"class": "ellipsis rank01"}) 
                            singers = parse.find_all("div", {"class": "ellipsis rank02"}) 
                            albums = parse.find_all("div",{"class": "ellipsis rank03"}) 
                            dates = driver.find_element(By.CLASS_NAME,'datelk').text

                            likes_elements = parse.find_all("span", {"class": "cnt"})  # 좋아요 수 추출
                            likes = [element.text.strip() for element in likes_elements]


                            rank = [i for i in range(1,101)]


                            for t in titles:
                                title.append(t.find('a').text) 
                            for s in singers:
                                singer.append(s.find('span', {"class": "checkEllipsis"}).text)

                            for a in albums:
                                album.append(a.find('a').text)
                                date.append(dates)
                            for l in likes:
                                l = (re.sub(r'\D', '', l.replace("총건수", "").strip()))
                                like.append(l)

                            time.sleep(1)
                        except NoSuchElementException:
                            print(f"{year}년 {month}월 {week}주차의 장르종합 데이터가 존재하지 않습니다.")
                            continue

                        
                        # 페이지 로딩과 서버 부하를 고려하여 적당한 대기 시간 추가
                        # WebDriverWait(driver, 10).until(EC.staleness_of(week_btn))
    data = {
    'rank' : rank * (len(title) // 100),
    'title': title,
    'singer': singer,
    'album': album,
    'dates': date,
    'like' : like, 
    }
    chart_df = pd.DataFrame(data)
    chart_df.to_csv('melon_chart_real.csv', index=False, encoding='utf-8-sig')
    print("종료.")
    driver.quit()

if __name__ == '__main__':
    main()



2024년 1월 5주차 데이터가 존재하지 않습니다.


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.107)
Stacktrace:
	GetHandleVerifier [0x00844CA3+225091]
	(No symbol) [0x00774DF1]
	(No symbol) [0x00619A7A]
	(No symbol) [0x005FE312]
	(No symbol) [0x0067517B]
	(No symbol) [0x006855A6]
	(No symbol) [0x0066F2F6]
	(No symbol) [0x006479B9]
	(No symbol) [0x0064879D]
	sqlite3_dbdata_init [0x00CB9A43+4064547]
	sqlite3_dbdata_init [0x00CC104A+4094762]
	sqlite3_dbdata_init [0x00CBB948+4072488]
	sqlite3_dbdata_init [0x009BC9A9+930953]
	(No symbol) [0x007807C4]
	(No symbol) [0x0077ACE8]
	(No symbol) [0x0077AE11]
	(No symbol) [0x0076CA80]
	BaseThreadInitThunk [0x76377BA9+25]
	RtlInitializeExceptionChain [0x7706BD2B+107]
	RtlClearBits [0x7706BCAF+191]


In [5]:
data = {
    'rank' : rank * (len(title) // 100),
    'title': title,
    'singer': singer,
    'album': album,
    'dates': date,
    'like' : like, 
    }
chart_df = pd.DataFrame(data)
chart_df.to_csv('melon_chart.csv', index=False, encoding='utf-8-sig')

NameError: name 'rank' is not defined